## [Creating a chatbot with Gradio, Llama 3.2 and Ollama API](https://pyimagesearch.com/2025/02/10/building-a-multimodal-gradio-chatbot-with-llama-3-2-using-the-ollama-api/)

- Gradio: enables create user-friendly and interactive web applications and simplifies create ML workflow
  - Multimodal support
  - Easily integrates with APIs (Ollama!)

---

- Ollama: enables run LLMs locally
  - Local execution
  - Model management
  - Two ways to interact: Restful APIs calls and Python clien

1. 

```python

curl http://localhost:11434/api/generate -d '{
  "model": "llama3.2",
  "prompt": "Describe the Eiffel Tower."
}'
```

2. 

```python
import ollama
# Initialize the client
client = ollama.Client()
# Send a chat request
response = client.chat(
    model="llama3.2",
    messages=[
        {"role": "user", "content": "Describe the Eiffel Tower."}
    ]
)
# Print the response
print(response['message']['content'])
```

- You can store LLMs on your local machine efficiently with Ollama in `~/.ollama/models`
- Pull models `ollama pull llama3.2-vision`
- `ollama list`
- `ollama remove llama3.2-vision`
- Ollama allows redirecting model storage to a different directory

--- 

- Llama3.2 (Meta) enables processing both text and images
  - Multimodal processing
  - Variants: 11B and 90B
    - Smaller variants are optimized to run locally!
  - What it cand do: Visual Question Answering (VQA), Image Captioning, Visual Reasoning

---

Versions: `ollama==0.3.3`, `pillow==11.0.0 (PIL)`, and `gradio==5.5.0`

In [29]:
pip install gradio==5.5.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install pillow==11.0.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [31]:
!pip install ollama==0.3.3

Defaulting to user installation because normal site-packages is not writeable


## Setting up the utilities

In [32]:
import io

# converts a PIL.Image object into a byte stream
def image_to_bytes(image):
  buffered = io.BytesIO()
  image.save(buffered, format="JPEG")
  return buffered.getvalue()

# why? APIs like Ollama require images to be sent as byte data instead of Python objects.

## Chatbot logic

In [33]:
# 'point point21321 point point'.strip('point point')
# '21321'

# '       a        '.strip()
# 'a'

In [34]:
import ollama
class ChatBot:
  def __init__(self, model_name, retries):
    self.client = ollama.Client()
    self.model_name = model_name
    self.retries = retries # nº retry attempts in case of API request failures
    self.history = []

  def add_user_message(self, text_input, image_input, response_style):
    message = {'role': 'user', 'content': text_input.strip()}
    
    if response_style == "Detailed":
      message['content'] += " Please provide a detailed response."
    elif response_style == "Concise":
      message['content'] += " Keep the response concise."
    self.history.append(message)

  def generate_response(self):
    for attempt in range(self.retries):
      response = self.client.chat(model=self.model_name, messages=self.history)

      assistant_message = {'role': 'assistant', 'content': response['message']['content']}
      return assistant_message

## Interface - Gradio

In [ ]:
import gradio as gr

MODEL_NAME = 'llama3.2-vision'
chatbot = ChatBot(MODEL_NAME, 5)

def handle_user_input(text_input, image_input, response_style):
  if not text_input and not image_input:
    return ("Please provide either text or an image.", 
            "\n".join(f"{msg['role'].capitalize()}: {msg['content']}" for msg in chatbot.history))
  
  try:
    generated_text = chatbot.generate_response()
    return generated_text, chatbot.history
  except Exception as e:
    return (f"An error occurred: {str(e)}", 
            "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in chatbot.history]))


In [39]:
with gr.Blocks() as demo:
  gr.Markdown("# Enhanced Multimodal Chatbot with Llama 3.2 Vision")
  gr.Markdown("Upload an image or enter a text prompt, choose a response style, and view the generated response along with the interaction history.")
  
  with gr.Row():
      text_input = gr.Textbox(lines=2, placeholder="Enter your question here...", label="Text Input")
      image_input = gr.Image(type="pil", label="Image Input (Optional)")
      response_style = gr.Dropdown(["Detailed", "Concise", "Creative"], label="Response Style", value="Detailed")
    
      with gr.Column():
        generated_response = gr.Textbox(label="Generated Response")
  
  submit_button = gr.Button("Submit")
  
  submit_button.click(
      fn=handle_user_input,
      inputs=[text_input, image_input, response_style],
      outputs=generated_response
  )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860


/home/ludmila/.local/lib/python3.10/site-packages/gradio/analytics.py:108: UserWarning: unable to parse version details from package URL.
  warnings.warn("unable to parse version details from package URL.")



Could not create share link. Missing file: /home/ludmila/.local/lib/python3.10/site-packages/gradio/frpc_linux_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.3
3. Move the file to this location: /home/ludmila/.local/lib/python3.10/site-packages/gradio
